In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import numpy as np

import torchvision
import torchvision.transforms as transforms

# Custom Dataset

- torchvision -> ImageFolder
- 원하는 형태의 데이터로 가공
- Custom Dataset의 구성

In [ ]:
trans = transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root="./data",
                                       train=True,
                                       download=True,
                                       transform=trans)

testset = torchvision.datasets.CIFAR10(root="./data",
                                      train=False,
                                      download=True,
                                      transform=trans)

trainloader = DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)


In [1]:
class my_dataset(touch.utils.data.Dataset):
    
    def __init__(self, x, transforms=None):
        
        # 데이터 선처리를 해준다
        
    def __len__(self):
        
        # 데이터셋의 길이를 반환한다
    
    def __getitem__(self, idx):
        
        # 데이터셋에서 한개의 데이터를 가져오는 함수
    

IndentationError: expected an indented block (<ipython-input-1-59dfa0b9ad09>, line 7)

- Network를 더 쉽게 만들기
- Model 저장하고 불러다가 다시 쓰기

In [5]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import numpy as np

import torchvision
import torchvision.transforms as transforms

import visdom

In [3]:
trans = transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root="./data",
                                       train=True,
                                       download=True,
                                       transform=trans)

testset = torchvision.datasets.CIFAR10(root="./data",
                                      train=False,
                                      download=True,
                                      transform=trans)


Files already downloaded and verified
Files already downloaded and verified


In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, 5),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 3),
            nn.Conv2d(128, 128, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3),
            nn.MaxPool2d(2)
        )
        
        self.layer2_1 = nn.Sequential(
            nn.Conv2d(256, 512, 7, 1, 2),
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )
             
        self.layer2_2 = nn.Sequential(
            nn.Conv2d(256, 512, 5, 1, 1),
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )        
        
        self.layer2_3 = nn.Sequential(
            nn.Conv2d(256, 512, 3),
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(3*64*4*4, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 10)
        )
    
    def forward(self, x):
        print(x.data.shape)
        x = self.layer1(x)
        x1 = self.layer2_1(x)
        x2 = self.layer2_2(x)
        x3 = self.layer2_3(x)
        
        # 1은 dimension을 의미.
        # x값의 형태(x.shape) == batch_size * Ch * H * W
        # 1이라고 하는 것은 "채널"을 기준으로 합치라는 의미.(x.shape의 인덱스)
        x = torch.cat((x1, x2, x3), 1)
        
        # 채널 이하의 것은 한줄로 펴라. (Height, Width)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        
        return x

In [15]:
a = torch.rand(1, 3, 32, 32)
print(a.shape)
a = Variable(a)
print(a.shape)

torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])


In [16]:
network = Net()
out = network(a)
print(out.shape)

torch.Size([1, 3, 32, 32])
torch.Size([1, 10])


In [17]:
# out의 value값.
print(out)

tensor([[ 0.0144,  0.0261, -0.0229, -0.0358,  0.0162, -0.0306,  0.0121, -0.0137,
          0.0285,  0.0013]], grad_fn=<AddmmBackward>)


In [18]:
# 네트워크 저장

torch.save(network.state_dict(), './cnn.pth')

In [20]:
model = Net()
model.load_state_dict(torch.load('./cnn.pth'))

<All keys matched successfully>

In [22]:
out = model(a)
print(out)

torch.Size([1, 3, 32, 32])
tensor([[ 0.0144,  0.0261, -0.0229, -0.0358,  0.0162, -0.0306,  0.0121, -0.0137,
          0.0285,  0.0013]], grad_fn=<AddmmBackward>)


In [23]:
print(out)

tensor([[ 0.0144,  0.0261, -0.0229, -0.0358,  0.0162, -0.0306,  0.0121, -0.0137,
          0.0285,  0.0013]], grad_fn=<AddmmBackward>)


## visdom

In [25]:
vis = visdom.Visdom()

textwindow = vis.text("Hello Pytorch")

Setting up a new session...


In [ ]:
for i in range(500):
    vis.line(Y=torch.rand(1), X=np.array([i+5]), win=plot, update="append")